In [1]:
from logic import *
import termcolor

def check_knowledge(knowledge):
    for symbol in symbols:
        if model_check(knowledge, symbol):
            termcolor.cprint(f"{symbol}: YES", "green")
        elif not model_check(knowledge, Not(symbol)):
            print(f"{symbol}: MAYBE")

# Harry
If it didn't rain, Harry visited Hagrid today. <br>
Harry visited Hagrid or Dumbledore today, but not both.<br>
Harry visited Dumbledore today.<br>
It rained today.<br>
Harry did not visit Hagrid today.

In [2]:
# we have three symbols: rain, hagrid, dumbledore
rain = Symbol("rain") # It is raining
hagrid = Symbol("hagrid") # Harry visited Hagrid
dumbledore = Symbol("dumbledore") # Harry visited Dumbledore

In [3]:
sentence = And(rain, hagrid)
print(sentence.formula())

rain ∧ hagrid


In [4]:
# If it didn't rain, Harry visited Hagrid today.
knowledge = Implication(Not(rain), hagrid)
print(knowledge.formula())

(¬rain) => hagrid


In [5]:
knowledge = And(
    Implication(Not(rain), hagrid), # If it didn't rain, Harry visited Hagrid today.
    # Harry visited Hagrid or Dumbledore today, but not both.
    Or(hagrid, dumbledore), Not(And(hagrid, dumbledore)),
    dumbledore # Harry visited Dumbledore today.
)

print(knowledge.formula())

((¬rain) => hagrid) ∧ (hagrid ∨  dumbledore) ∧ (¬(hagrid ∧ dumbledore)) ∧ dumbledore


# Clue

In [6]:
mustard = Symbol("ColMustard")
plum = Symbol("ProfPlum")
scarlet = Symbol("MsScarlet")
characters = [mustard, plum, scarlet]

ballroom = Symbol("ballroom")
kitchen = Symbol("kitchen")
library = Symbol("library")
rooms = [ballroom, kitchen, library]

knife = Symbol("knife")
revolver = Symbol("revolver")
wrench = Symbol("wrench")
weapons = [knife, revolver, wrench]

symbols = characters + rooms + weapons

In [7]:
# There must be a person, room, and weapon.
# the killer, the roon, and the weapon cards are inside an envelope
knowledge = And(
    Or(mustard, plum, scarlet), # one of these people is the killer
    Or(ballroom, kitchen, library), # one of these rooms is the scene of the crime
    Or(knife, revolver, wrench) # one of these weapons was used to commit the crime
)

print(knowledge.formula())

(ColMustard ∨  ProfPlum ∨  MsScarlet) ∧ (ballroom ∨  kitchen ∨  library) ∧ (knife ∨  revolver ∨  wrench)


In [8]:
check_knowledge(knowledge)

ColMustard: MAYBE
ProfPlum: MAYBE
MsScarlet: MAYBE
ballroom: MAYBE
kitchen: MAYBE
library: MAYBE
knife: MAYBE
revolver: MAYBE
wrench: MAYBE


In [9]:
# cards in hand
knowledge.add(And(
    Not(mustard), Not(kitchen), Not(revolver)
))

# if cards in hand, they are not in the envelope
check_knowledge(knowledge)

ProfPlum: MAYBE
MsScarlet: MAYBE
ballroom: MAYBE
library: MAYBE
knife: MAYBE
wrench: MAYBE


In [10]:
# from another player's guess, we know at least one of these are not in the envelope
knowledge.add(Or(
    Not(scarlet), Not(library), Not(wrench)
))

check_knowledge(knowledge)

ProfPlum: MAYBE
MsScarlet: MAYBE
ballroom: MAYBE
library: MAYBE
knife: MAYBE
wrench: MAYBE


In [11]:
# card is shown by another player
knowledge.add(Not(plum))

# we know the only possible killer is Ms. Scarlet
check_knowledge(knowledge)

MsScarlet: YES
ballroom: MAYBE
library: MAYBE
knife: MAYBE
wrench: MAYBE


In [12]:
# card is shown by another player
knowledge.add(Not(ballroom))

# we know the only possible location is library
check_knowledge(knowledge)

MsScarlet: YES
library: YES
knife: YES


from Or[ Not(scarlet), Not(library), Not(wrench) ], we know it is not(wrench). Then the weapon must be knife.

# Puzzle
• Gilderoy, Minerva, Pomona and Horace each belong to a different one of the four houses: Gryffindor, Hufflepuff, Ravenclaw, and Slytherin House. <br>
• Gilderoy belongs to Gryffindor or Ravenclaw. <br>
• Pomona does not belong in Slytherin. <br>
• Minerva belongs to Gryffindor.

In [13]:
people = ["Gilderoy", "Pomona", "Minerva", "Horace"]
houses = ["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]

symbols = []

knowledge = And()

for person in people:
    for house in houses:
        symbols.append(Symbol(f"{person}{house}"))

# print all symbols
for i in range(1,17):
    if i%4 != 0:
        print(symbols[i-1], end=", ")
    else:
        print(symbols[i-1], end="\n")

GilderoyGryffindor, GilderoyHufflepuff, GilderoyRavenclaw, GilderoySlytherin
PomonaGryffindor, PomonaHufflepuff, PomonaRavenclaw, PomonaSlytherin
MinervaGryffindor, MinervaHufflepuff, MinervaRavenclaw, MinervaSlytherin
HoraceGryffindor, HoraceHufflepuff, HoraceRavenclaw, HoraceSlytherin


In [14]:
# Each person belongs to a house.
for person in people:
    knowledge.add(Or(
        Symbol(f"{person}Gryffindor"),
        Symbol(f"{person}Hufflepuff"),
        Symbol(f"{person}Ravenclaw"),
        Symbol(f"{person}Slytherin")
    ))

check_knowledge(knowledge)

GilderoyGryffindor: MAYBE
GilderoyHufflepuff: MAYBE
GilderoyRavenclaw: MAYBE
GilderoySlytherin: MAYBE
PomonaGryffindor: MAYBE
PomonaHufflepuff: MAYBE
PomonaRavenclaw: MAYBE
PomonaSlytherin: MAYBE
MinervaGryffindor: MAYBE
MinervaHufflepuff: MAYBE
MinervaRavenclaw: MAYBE
MinervaSlytherin: MAYBE
HoraceGryffindor: MAYBE
HoraceHufflepuff: MAYBE
HoraceRavenclaw: MAYBE
HoraceSlytherin: MAYBE


In [15]:
# Only one house per person.
for person in people:
    for h1 in houses:
        for h2 in houses:
            if h1 != h2:
                knowledge.add(
                    Implication(Symbol(f"{person}{h1}"), Not(Symbol(f"{person}{h2}")))
                )

check_knowledge(knowledge)

GilderoyGryffindor: MAYBE
GilderoyHufflepuff: MAYBE
GilderoyRavenclaw: MAYBE
GilderoySlytherin: MAYBE
PomonaGryffindor: MAYBE
PomonaHufflepuff: MAYBE
PomonaRavenclaw: MAYBE
PomonaSlytherin: MAYBE
MinervaGryffindor: MAYBE
MinervaHufflepuff: MAYBE
MinervaRavenclaw: MAYBE
MinervaSlytherin: MAYBE
HoraceGryffindor: MAYBE
HoraceHufflepuff: MAYBE
HoraceRavenclaw: MAYBE
HoraceSlytherin: MAYBE


In [16]:
# Only one person per house.
for house in houses:
    for p1 in people:
        for p2 in people:
            if p1 != p2:
                knowledge.add(
                    Implication(Symbol(f"{p1}{house}"), Not(Symbol(f"{p2}{house}")))
                )

check_knowledge(knowledge)

GilderoyGryffindor: MAYBE
GilderoyHufflepuff: MAYBE
GilderoyRavenclaw: MAYBE
GilderoySlytherin: MAYBE
PomonaGryffindor: MAYBE
PomonaHufflepuff: MAYBE
PomonaRavenclaw: MAYBE
PomonaSlytherin: MAYBE
MinervaGryffindor: MAYBE
MinervaHufflepuff: MAYBE
MinervaRavenclaw: MAYBE
MinervaSlytherin: MAYBE
HoraceGryffindor: MAYBE
HoraceHufflepuff: MAYBE
HoraceRavenclaw: MAYBE
HoraceSlytherin: MAYBE


In [17]:
print(knowledge.formula())

(GilderoyGryffindor ∨  GilderoyHufflepuff ∨  GilderoyRavenclaw ∨  GilderoySlytherin) ∧ (PomonaGryffindor ∨  PomonaHufflepuff ∨  PomonaRavenclaw ∨  PomonaSlytherin) ∧ (MinervaGryffindor ∨  MinervaHufflepuff ∨  MinervaRavenclaw ∨  MinervaSlytherin) ∧ (HoraceGryffindor ∨  HoraceHufflepuff ∨  HoraceRavenclaw ∨  HoraceSlytherin) ∧ (GilderoyGryffindor => (¬GilderoyHufflepuff)) ∧ (GilderoyGryffindor => (¬GilderoyRavenclaw)) ∧ (GilderoyGryffindor => (¬GilderoySlytherin)) ∧ (GilderoyHufflepuff => (¬GilderoyGryffindor)) ∧ (GilderoyHufflepuff => (¬GilderoyRavenclaw)) ∧ (GilderoyHufflepuff => (¬GilderoySlytherin)) ∧ (GilderoyRavenclaw => (¬GilderoyGryffindor)) ∧ (GilderoyRavenclaw => (¬GilderoyHufflepuff)) ∧ (GilderoyRavenclaw => (¬GilderoySlytherin)) ∧ (GilderoySlytherin => (¬GilderoyGryffindor)) ∧ (GilderoySlytherin => (¬GilderoyHufflepuff)) ∧ (GilderoySlytherin => (¬GilderoyRavenclaw)) ∧ (PomonaGryffindor => (¬PomonaHufflepuff)) ∧ (PomonaGryffindor => (¬PomonaRavenclaw)) ∧ (PomonaGryffindor => 

In [18]:
# Gilderoy is in Gryffindor or Ravenclaw.
knowledge.add(
    Or(Symbol("GilderoyGryffindor"), Symbol("GilderoyRavenclaw"))
)

check_knowledge(knowledge)

GilderoyGryffindor: MAYBE
GilderoyRavenclaw: MAYBE
PomonaGryffindor: MAYBE
PomonaHufflepuff: MAYBE
PomonaRavenclaw: MAYBE
PomonaSlytherin: MAYBE
MinervaGryffindor: MAYBE
MinervaHufflepuff: MAYBE
MinervaRavenclaw: MAYBE
MinervaSlytherin: MAYBE
HoraceGryffindor: MAYBE
HoraceHufflepuff: MAYBE
HoraceRavenclaw: MAYBE
HoraceSlytherin: MAYBE


In [19]:
# Pomona is not in Slytherin.
knowledge.add(
    Not(Symbol("PomonaSlytherin"))
)

check_knowledge(knowledge)

GilderoyGryffindor: MAYBE
GilderoyRavenclaw: MAYBE
PomonaGryffindor: MAYBE
PomonaHufflepuff: MAYBE
PomonaRavenclaw: MAYBE
MinervaGryffindor: MAYBE
MinervaHufflepuff: MAYBE
MinervaRavenclaw: MAYBE
MinervaSlytherin: MAYBE
HoraceGryffindor: MAYBE
HoraceHufflepuff: MAYBE
HoraceRavenclaw: MAYBE
HoraceSlytherin: MAYBE


In [20]:
# Minerva is in Gryffindor.
knowledge.add(
    Symbol("MinervaGryffindor")
)

check_knowledge(knowledge)

GilderoyRavenclaw: YES
PomonaHufflepuff: YES
MinervaGryffindor: YES
HoraceSlytherin: YES


# Mastermind

In [22]:
colors = ["red", "blue", "green", "yellow"]
symbols = []
for i in range(4):
    for color in colors:
        symbols.append(Symbol(f"{color}{i}"))

knowledge = And()

# Each color has a position.
for color in colors:
    knowledge.add(Or(
        Symbol(f"{color}0"),
        Symbol(f"{color}1"),
        Symbol(f"{color}2"),
        Symbol(f"{color}3")
    ))

# print all symbols
for i in range(1,17):
    if i%4 != 0:
        print(symbols[i-1], end=", ")
    else:
        print(symbols[i-1], end="\n")

red0, blue0, green0, yellow0
red1, blue1, green1, yellow1
red2, blue2, green2, yellow2
red3, blue3, green3, yellow3


In [23]:
# Only one position per color.
for color in colors:
    for i in range(4):
        for j in range(4):
            if i != j:
                knowledge.add(Implication(
                    Symbol(f"{color}{i}"), Not(Symbol(f"{color}{j}"))
                ))

check_knowledge(knowledge)

red0: MAYBE
blue0: MAYBE
green0: MAYBE
yellow0: MAYBE
red1: MAYBE
blue1: MAYBE
green1: MAYBE
yellow1: MAYBE
red2: MAYBE
blue2: MAYBE
green2: MAYBE
yellow2: MAYBE
red3: MAYBE
blue3: MAYBE
green3: MAYBE
yellow3: MAYBE


In [24]:
# Only one color per position.
for i in range(4):
    for c1 in colors:
        for c2 in colors:
            if c1 != c2:
                knowledge.add(Implication(
                    Symbol(f"{c1}{i}"), Not(Symbol(f"{c2}{i}"))
                ))

check_knowledge(knowledge)

red0: MAYBE
blue0: MAYBE
green0: MAYBE
yellow0: MAYBE
red1: MAYBE
blue1: MAYBE
green1: MAYBE
yellow1: MAYBE
red2: MAYBE
blue2: MAYBE
green2: MAYBE
yellow2: MAYBE
red3: MAYBE
blue3: MAYBE
green3: MAYBE
yellow3: MAYBE


In [25]:
knowledge.add(Or(
    And(Symbol("red0"), Symbol("blue1"), Not(Symbol("green2")), Not(Symbol("yellow3"))),
    And(Symbol("red0"), Symbol("green2"), Not(Symbol("blue1")), Not(Symbol("yellow3"))),
    And(Symbol("red0"), Symbol("yellow3"), Not(Symbol("blue1")), Not(Symbol("green2"))),
    And(Symbol("blue1"), Symbol("green2"), Not(Symbol("red0")), Not(Symbol("yellow3"))),
    And(Symbol("blue1"), Symbol("yellow3"), Not(Symbol("red0")), Not(Symbol("green2"))),
    And(Symbol("green2"), Symbol("yellow3"), Not(Symbol("red0")), Not(Symbol("blue1")))
))

check_knowledge(knowledge)

red0: MAYBE
blue0: MAYBE
green0: MAYBE
yellow0: MAYBE
red1: MAYBE
blue1: MAYBE
green1: MAYBE
yellow1: MAYBE
red2: MAYBE
blue2: MAYBE
green2: MAYBE
yellow2: MAYBE
red3: MAYBE
blue3: MAYBE
green3: MAYBE
yellow3: MAYBE


In [26]:
knowledge.add(And(
    Not(Symbol("blue0")),
    Not(Symbol("red1")),
    Not(Symbol("green2")),
    Not(Symbol("yellow3"))
))

check_knowledge(knowledge)

red0: YES
blue1: YES
yellow2: YES
green3: YES


Model checking's time complexity is 2^n